In [1]:
import pandas as pd
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from IPython.display import display
from IPython.display import display_html



# Load the data
data=pd.read_excel('F1 RACES 2021.xlsx',index_col=None,header=None)
races=data.iloc[1:,2:].reset_index(drop=True)
m=races.shape[0]
n=races.shape[1]
races.set_axis(list(range(n)), axis=1, inplace=True)
races=np.array(races)

# Who beat who matrix

In [2]:
scores=np.zeros((m,m))
teams_score=np.zeros(10)


# Here we create a matrix of 'who beat who'. Every driver   
# receives 1 point for any driver finished in a lower position 
for i in range(m):
    for j in range(i,m):
        if i==j:
            scores[i,j]=0
        else:
            scores[i,j]=len(np.where(races[i,:]<races[j,:])[0])
            scores[j,i]=n-scores[i,j]
            
driver_score=scores.astype('int') # keep every driver's points array, for each Grand Prix 

# Calculate a matrix of teams' scores by summing the points of each team's driver
teams=['Alfa Romeo Racing Ferrari', 'AlphaTauri Honda', 'Alpine Renault', 'Aston Martin Mercedes', 'Ferrari', 
       'Haas Ferrari', 'McLaren Mercedes', 'Mercedes', 'Red Bull Racing Honda', 'Williams Mercedes']

for i in range(len(teams)):
    for j in range(1,m+1):
        if data.iloc[j,1]==teams[i]:
            teams_score[i] += np.sum(scores[j-1,:])
teams_results={}
for i in range(len(teams)):
    teams_results.update({teams[i]:teams_score[i]})

In [3]:
teams_score = teams_score.astype('int')

# PageRank algorithm

In [4]:
# The PageRank Method for drivers points
for i in range(m):
    scores[i,:]=scores[i,:]/np.sum(scores[i,:])
a=0.85
G=a*scores+(1-a)*(np.ones((m,m))/m)


k=0
E,V=np.linalg.eig(G)
x=np.ones((21))
old_eig_value=0
# Find the nomber k of iterations necessary to reach convergence
for i in range(100):
    x=np.dot(G,x)
    eig_value=np.linalg.norm(x)
    error=np.abs((eig_value-old_eig_value)/eig_value)
    x=x/eig_value
    if error<0.05:
        break
    old_eig_value=eig_value
    k+=1   

q=(np.ones((1,m)))/m@ G**k

In [5]:
# Calculate and display the final standings of both teams and drivers
q=q.T
names=data.iloc[1:,0]
names=names.to_numpy()
results={}
for i in range(m):
    results.update({names[i]:q[i]})

# Web scraping F1 drivers' final rankings

In [16]:
import requests
# Scaping driver data
driver_standings_url = 'https://www.formula1.com/en/results.html/2021/drivers.html'
driver_url_data = requests.get(driver_standings_url)

from bs4 import BeautifulSoup
driver_soup = BeautifulSoup(driver_url_data.text)
table = driver_soup.select('table.resultarchive-table')
drivers_table=pd.read_html(driver_url_data.text, index_col=1)[0]

In [17]:
drivers_table

,Unnamed: 0,Driver,Nationality,Car,PTS,Unnamed: 6
Pos,,,,,,
1,NaN,Max Verstappen VER,NED,Red Bull Racing Honda,395.5,NaN
2,NaN,Lewis Hamilton HAM,GBR,Mercedes,387.5,NaN
3,NaN,Valtteri Bottas BOT,FIN,Mercedes,226.0,NaN
4,NaN,Sergio Perez PER,MEX,Red Bull Racing Honda,190.0,NaN
5,NaN,Carlos Sainz SAI,ESP,Ferrari,164.5,NaN
6,NaN,Lando Norris NOR,GBR,McLaren Mercedes,160.0,NaN
7,NaN,Charles Leclerc LEC,MON,Ferrari,159.0,NaN
8,NaN,Daniel Ricciardo RIC,AUS,McLaren Mercedes,115.0,NaN
9,NaN,Pierre Gasly GAS,FRA,AlphaTauri Honda,110.0,NaN


In [18]:
drivers_table = drivers_table.drop(['Unnamed: 0','Nationality','Car','Unnamed: 6'], axis=1) #remove unnamed, Nationality & car columns
drivers_table['Driver'] = drivers_table['Driver'].map(lambda x: str(x)[:-4])
drivers_table.at[16, 'Driver'] = 'Kimi Raikkonen'   # set Kimi's name right
drivers_table['PTS'] = drivers_table['PTS'].apply(lambda x: '%.1f' % x)

# World Drivers' Championship standings

In [19]:
final_results=dict(sorted(results.items(), key=lambda x:x[1]))

total_points = sorted(np.sum(driver_score, axis=1).tolist(), reverse = True) #total driver's sum of points 

a1=np.arange(1,m+1)
drivers_data=pd.DataFrame({'Pos':list(a1), 'Driver':final_results.keys(), 'PTS':total_points}).set_index('Pos')

In [20]:
drivers_table = drivers_table.style.set_table_attributes("style='display:inline; margin-right:100px;'").set_caption('F1 2021 Drivers\' Standings - '
                                                         'Real Data').set_table_styles([{'selector':'caption', 
'props':[('font-size', '14px'), ('text-align','center'), ('color','black'), ('font-style', 'italic'), ('font-weight', 'bold')] }])
drivers_data = drivers_data.style.set_table_attributes("style='display:inline'").set_caption('F1 2021 Drivers\' '
                                                                                             'Standings - PageRank').set_table_styles([{'selector':'caption', 
'props':[('font-size', '14px'), ('text-align','center'), 
         ('color','black'), ('font-style', 'italic'), ('font-weight', 'bold')] }])

In [21]:
display_html(drivers_table._repr_html_() + drivers_data._repr_html_(), raw=True)

,Driver,PTS
Pos,,
1,Max Verstappen,395.5
2,Lewis Hamilton,387.5
3,Valtteri Bottas,226.0
4,Sergio Perez,190.0
5,Carlos Sainz,164.5
6,Lando Norris,160.0
7,Charles Leclerc,159.0
8,Daniel Ricciardo,115.0
9,Pierre Gasly,110.0


# Web scraping F1 Constructors' final rankings

In [22]:
# Scaping teams' data
teams_standings_url = 'https://www.formula1.com/en/results.html/2021/team.html'
teams_url_data = requests.get(teams_standings_url)

from bs4 import BeautifulSoup
teams_soup = BeautifulSoup(teams_url_data.text)
tm_table = teams_soup.select('table.resultarchive-table')
teams_table=pd.read_html(teams_url_data.text, index_col=1)[0]

In [23]:
teams_table

,Unnamed: 0,Team,PTS,Unnamed: 4
Pos,,,,
1,NaN,Mercedes,613.5,NaN
2,NaN,Red Bull Racing Honda,585.5,NaN
3,NaN,Ferrari,323.5,NaN
4,NaN,McLaren Mercedes,275.0,NaN
5,NaN,Alpine Renault,155.0,NaN
6,NaN,AlphaTauri Honda,142.0,NaN
7,NaN,Aston Martin Mercedes,77.0,NaN
8,NaN,Williams Mercedes,23.0,NaN
9,NaN,Alfa Romeo Racing Ferrari,13.0,NaN


In [24]:
teams_table = teams_table.drop(['Unnamed: 0','Unnamed: 4'], axis=1) #remove unnamed columns
teams_table['PTS'] = teams_table['PTS'].apply(lambda x: '%.1f' % x) #format points with 1 decimal digit

# World Constructors' Championship standings

In [25]:
final_teams=dict(sorted(teams_results.items(), key=lambda x:x[1], reverse=True))
team_points = sorted(teams_score.tolist(), reverse = True)

teams_data=pd.DataFrame({'Pos':list(np.arange(1,11)), 'Team':final_teams.keys(), 
                         'PTS':team_points}).set_index('Pos')
teams_data=teams_data.style.set_table_attributes("style='display:inline; margin-right:100px;'").set_caption('F1 2021 Constructors\' Standings - '
                                                         'PageRank').set_table_styles([{'selector':'caption', 
'props':[('font-size', '14px'), ('text-align','center'), ('color','black'), ('font-style', 'italic'), ('font-weight', 'bold')] }])

teams_table = teams_table.style.set_table_attributes("style='display:inline; margin-right:100px;'").set_caption('F1 2021 Constructors\' Standings - '
                                                         'Real Data').set_table_styles([{'selector':'caption', 
'props':[('font-size', '14px'), ('text-align','center'), ('color','black'), ('font-style', 'italic'), ('font-weight', 'bold')] }])

In [26]:
display_html(teams_table._repr_html_() + teams_data._repr_html_(), raw=True)

,Team,PTS
Pos,,
1,Mercedes,613.5
2,Red Bull Racing Honda,585.5
3,Ferrari,323.5
4,McLaren Mercedes,275.0
5,Alpine Renault,155.0
6,AlphaTauri Honda,142.0
7,Aston Martin Mercedes,77.0
8,Williams Mercedes,23.0
9,Alfa Romeo Racing Ferrari,13.0
